# Playground for Riot Watcher

This is a testing notebook where I explore and test out features of `riotwatcher`. 

In [11]:
from riotwatcher import LolWatcher, ApiError
import pandas as pd
import zilean
from zilean.snapshots import SnapShots

import json

api_key = zilean.read_api_key()
watcher = LolWatcher(api_key=api_key)
my_region = 'na1'
me = watcher.summoner.by_name(my_region, 'yENatividad')

ImportError: cannot import name 'SnapShots' from 'zilean' (/Users/JDDJ/Documents/Self/Programming/leagueoflegends/zilean/zilean/__init__.py)

## Match Timeline Summary

In [2]:
my_matches = watcher.match.matchlist_by_puuid(my_region, me['puuid'])
latest_matchid = my_matches[2]
latest_match = watcher.match.by_id(my_region, latest_matchid)
latest_timeline = watcher.match.timeline_by_match(my_region, latest_matchid)

In [5]:
summary = zilean.process_timeframe(latest_timeline, [8, 10], latest_matchid)
pd.DataFrame([summary])

,level_0_frame8,timeEnemySpentControlled_0_frame8,totalGold_0_frame8,xp_0_frame8,totalDamageDone_0_frame8,totalDamageDoneToChampions_0_frame8,totalDamageTaken_0_frame8,creepScore_0_frame8,level_1_frame8,timeEnemySpentControlled_1_frame8,...,level_4_frame10,timeEnemySpentControlled_4_frame10,totalGold_4_frame10,xp_4_frame10,totalDamageDone_4_frame10,totalDamageDoneToChampions_4_frame10,totalDamageTaken_4_frame10,creepScore_4_frame10,matchId,win
0,-1,2121,-854,-855,-5051,-1807,1246,-27,0,120691,...,0,-20745,-457,237,2448,552,318,7,NA1_4307172207,False


In [6]:
snap = SnapShots(latest_timeline, [8, 10], latest_matchid)

NameError: name 'SnapShots' is not defined